In [ ]:
import os
import json
import numpy as np
from scipy.ndimage import uniform_filter1d
from google.colab import drive

# ✅ 挂载 Google Drive
drive.mount('/content/drive')

# ✅ 参数配置
start_index = 700
end_index   = 731
range_tag   = f"{start_index}-{end_index}"

BASE_PATH         = "/content/drive/MyDrive/Cluster-proj"
LOGITS_PATH       = f"{BASE_PATH}/output/llm_steps/whole_logits/deepseek7b-gsm-{range_tag}-hidden.json"
ERROR_INDEX_PATH  = f"{BASE_PATH}/output/error_index/{range_tag}_hidden_index.json"

# ✅ 输出目录
OUT_DIR = f"{BASE_PATH}/output/gated_hidden_output/{range_tag}"
os.makedirs(OUT_DIR, exist_ok=True)

# ✅ 加载数据
with open(LOGITS_PATH, "r") as f:
    logits_data = json.load(f)
with open(ERROR_INDEX_PATH, "r") as f:
    error_index_data = json.load(f)


Mounted at /content/drive


In [ ]:

# ✅ ReLU 门控函数（静态阈值）
def static_relu_gate(H, tau=0.6):
    return np.maximum(0.0, H - tau)

# ✅ 动态滑动门控
def dynamic_threshold_gate(H, window=5, k=1.0):
    H = np.asarray(H)
    mu = uniform_filter1d(H, size=window, mode="nearest")
    sigma = np.sqrt(uniform_filter1d((H - mu)**2, size=window, mode="nearest"))
    tau = mu + k * sigma
    gate = np.maximum(0.0, H - tau)
    return gate, tau

# ✅ 加权函数
def apply_gate_to_hidden(gate, hidden):
    gate = np.asarray(gate).reshape(-1, 1)  # shape (N, 1)
    hidden = np.asarray(hidden)            # shape (N, D)
    return gate * hidden                   # shape (N, D)

# ✅ 批处理
num_saved = 0
for qid in logits_data:
    for sid in logits_data[qid]:
        if sid.startswith("sampling"):
            try:
                token_probs = logits_data[qid][sid]["token_probs"]
                entropy = np.array([tok["topk_info"]["entropy"] for tok in token_probs])
                hidden  = np.array([tok["hidden_vector"] for tok in token_probs])  # (N, D)

                # ✅ 静态门控
                relu_gate = static_relu_gate(entropy, tau=0.6)
                gated_relu = apply_gate_to_hidden(relu_gate, hidden)

                # ✅ 动态门控
                dyn_gate, dyn_tau = dynamic_threshold_gate(entropy, window=5, k=1.0)
                gated_dyn = apply_gate_to_hidden(dyn_gate, hidden)

                # ✅ 保存为 .npy
                save_name = f"{qid}_{sid}"
                np.save(os.path.join(OUT_DIR, f"{save_name}_gated_relu.npy"), gated_relu)
                np.save(os.path.join(OUT_DIR, f"{save_name}_gated_dyn.npy"), gated_dyn)
                np.save(os.path.join(OUT_DIR, f"{save_name}_entropy.npy"), entropy)
                num_saved += 1

            except Exception as e:
                print(f"[ERROR] Skip {qid} {sid}: {e}")

print(f"✅ 已处理并保存 {num_saved} 个 (qid, sid) 对象到 {OUT_DIR}")


/tmp/ipython-input-2-1934557048.py:9: RuntimeWarning: invalid value encountered in sqrt
  sigma = np.sqrt(uniform_filter1d((H - mu)**2, size=window, mode="nearest"))


✅ 已处理并保存 93 个 (qid, sid) 对象到 /content/drive/MyDrive/Cluster-proj/output/gated_hidden_output/700-731


In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from tqdm import tqdm

# ✅ 配置路径
range_tag = "700-731"
BASE_PATH = "/content/drive/MyDrive/Cluster-proj"
DATA_PATH = f"{BASE_PATH}/output/gated_hidden_output/{range_tag}"
ERROR_INDEX_PATH = f"{BASE_PATH}/output/error_index/{range_tag}_hidden_index.json"
OUT_PATH = f"{BASE_PATH}/output/gated_hidden_vis_pair/{range_tag}"
os.makedirs(OUT_PATH, exist_ok=True)

# ✅ 加载错误 index，生成正负样本配对列表
with open(ERROR_INDEX_PATH, "r") as f:
    error_index_data = json.load(f)

paired_qid_sids = []
for qid, sid_dict in error_index_data.items():
    all_sids = {"sampling0", "sampling1", "sampling2"}
    error_sids = set(sid_dict.keys())
    correct_sids = all_sids - error_sids
    for neg_sid in error_sids:
        for pos_sid in correct_sids:
            paired_qid_sids.append((qid, neg_sid, pos_sid))

print(f"✅ 共发现 {len(paired_qid_sids)} 对正负样本")

# ✅ 逐对绘图
for qid, neg_sid, pos_sid in tqdm(paired_qid_sids):
    try:
        prefix_neg = f"{qid}_{neg_sid}"
        prefix_pos = f"{qid}_{pos_sid}"

        vec_neg = np.load(f"{DATA_PATH}/{prefix_neg}_gated_dyn.npy")
        vec_pos = np.load(f"{DATA_PATH}/{prefix_pos}_gated_dyn.npy")

        entropy_neg = np.load(f"{DATA_PATH}/{prefix_neg}_entropy.npy")
        entropy_pos = np.load(f"{DATA_PATH}/{prefix_pos}_entropy.npy")

        if len(vec_neg) < 4 or len(vec_pos) < 4:
            continue

        # 激活强度
        strength_neg = np.linalg.norm(vec_neg, axis=1)
        strength_pos = np.linalg.norm(vec_pos, axis=1)

        # 合并 PCA
        vec_all = np.concatenate([vec_neg, vec_pos], axis=0)
        pca = PCA(n_components=2)
        vec_all_2d = pca.fit_transform(vec_all)
        N_neg = len(vec_neg)

        vec2d_neg = vec_all_2d[:N_neg]
        vec2d_pos = vec_all_2d[N_neg:]

        # ✅ 可视化
        plt.figure(figsize=(8, 6))
        sc1 = plt.scatter(vec2d_neg[:, 0], vec2d_neg[:, 1], c=strength_neg, cmap='Reds', label="Negative", s=30, alpha=0.8, marker='o')
        sc2 = plt.scatter(vec2d_pos[:, 0], vec2d_pos[:, 1], c=strength_pos, cmap='Greens', label="Positive", s=30, alpha=0.8, marker='^')

        plt.title(f"{qid} | {neg_sid} (red) vs {pos_sid} (green)")
        plt.xlabel("PCA-1")
        plt.ylabel("PCA-2")
        plt.grid(True)
        plt.legend()
        plt.colorbar(sc1, label="Activation (Negative)", location='left', shrink=0.6)
        plt.colorbar(sc2, label="Activation (Positive)", location='right', shrink=0.6)
        plt.tight_layout()

        # ✅ 保存图像
        fname = f"{qid}_{neg_sid}_vs_{pos_sid}_pair.png"
        plt.savefig(os.path.join(OUT_PATH, fname))
        plt.close()
    except Exception as e:
        print(f"[ERROR] {qid} {neg_sid} vs {pos_sid}: {e}")

print(f"🎯 所有对比图保存至：{OUT_PATH}")


✅ 共发现 38 对正负样本


  3%|▎         | 1/38 [00:00<00:06,  5.53it/s]

[ERROR] q_700 sampling0 vs sampling1: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values


  8%|▊         | 3/38 [00:00<00:03, 10.66it/s]

[ERROR] q_700 sampling2 vs sampling1: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values
[ERROR] q_701 sampling1 vs sampling0: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the 

...

[ERROR] q_721 sampling2 vs sampling0: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values


 87%|████████▋ | 33/38 [00:07<00:00,  5.84it/s]

[ERROR] q_723 sampling2 vs sampling0: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values
[ERROR] q_723 sampling2 vs sampling1: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the 

100%|██████████| 38/38 [00:07<00:00,  5.24it/s]

[ERROR] q_727 sampling1 vs sampling2: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values
[ERROR] q_730 sampling2 vs sampling0: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the 